In [4]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [47]:
# Load API Credentials
with open('/Users/christianrim/.secret/yelp_api.json') as f:   #use your path here!
    login = json.load(f)

In [48]:
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)
yelp_api

In [49]:
# set our API call parameters 
LOCATION = '3744 W 6th St'
TERM = 'Korean'

In [24]:
# create a JSON_FILE filename
# include the search terms 
JSON_FILE = "Data/results_in_progress_korean_food.json"
JSON_FILE

'Data/results_in_progress_korean_food.json'

In [25]:
## checking if the file is there 
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    
    
    # save an empty list
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
# If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/results_in_progress_korean_food.json already exists.


Determines how many results are already in the file

In [26]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 0 previous results found.


Figure out how many pages of results we will need 

In [27]:
# use our yelp_api variable's search_query
results = yelp_api.search_query(location=LOCATION,
                               term=TERM,
                               offset=n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [28]:
## how many results total?
total_results = results['total']
total_results

969

In [29]:
# how many did we get the details for? 
results_per_page = len(results['businesses'])
results_per_page

20

From our queries, there are 969 businesses to retrieve from our API and we get 20 results at a time. 
- Can calculate the number of results remaining by subtracting our offset from our total
- can determine how many pages we will need to divide the results by 20


In [30]:
import time, math
# use math.cell to round up for the total number of pages of results
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

49

In [31]:
# join new results with old list with extend and save to file
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)

In [32]:
from tqdm.notebook import tqdm_notebook
import time
for i in tqdm_notebook(range(n_pages)):
    # adds 200 ms pause
    time.sleep(.2) 

  0%|          | 0/49 [00:00<?, ?it/s]

In [33]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    # add a 200ms pause
    time.sleep(.2)

  0%|          | 0/49 [00:00<?, ?it/s]

## Converting JSON to dataframe

In [34]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,h1R2iKYdm2lwukzMJvJqDw,hangari-kalguksu-los-angeles-4,Hangari Kalguksu,https://s3-media3.fl.yelpcdn.com/bphoto/X_U65O...,False,https://www.yelp.com/biz/hangari-kalguksu-los-...,2385,"[{'alias': 'korean', 'title': 'Korean'}, {'ali...",4.5,"{'latitude': 34.0628602582049, 'longitude': -1...","[pickup, delivery]",$$,"{'address1': '3470 W 6th St', 'address2': 'Ste...",+12133882326,(213) 388-2326,584.192122
1,qAAoilHU25Qr45FOQlA19g,kang-ho-dong-baekjeong-los-angeles-4,Kang Ho-dong Baekjeong,https://s3-media1.fl.yelpcdn.com/bphoto/0LQSEL...,False,https://www.yelp.com/biz/kang-ho-dong-baekjeon...,4902,"[{'alias': 'bbq', 'title': 'Barbeque'}, {'alia...",4.5,"{'latitude': 34.06375298900917, 'longitude': -...","[delivery, pickup]",$$$,"{'address1': '3465 W 6th St', 'address2': 'Ste...",+12133849678,(213) 384-9678,607.771480
2,WO12S4gvOYkYF8_70RWrQA,haneuem-los-angeles-3,HanEuem,https://s3-media3.fl.yelpcdn.com/bphoto/Rh_Q54...,False,https://www.yelp.com/biz/haneuem-los-angeles-3...,190,"[{'alias': 'korean', 'title': 'Korean'}, {'ali...",4.5,"{'latitude': 34.06427634725579, 'longitude': -...","[delivery, pickup]",NaN,"{'address1': '539 S Western Ave', 'address2': ...",+12133888988,(213) 388-8988,527.290392
3,uzAbw27XQTXTivjgf2bN2w,han-bat-sul-lung-tang-los-angeles-2,Han Bat Sul Lung Tang,https://s3-media2.fl.yelpcdn.com/bphoto/YttPox...,False,https://www.yelp.com/biz/han-bat-sul-lung-tang...,2666,"[{'alias': 'korean', 'title': 'Korean'}, {'ali...",4.5,"{'latitude': 34.065414, 'longitude': -118.3095...",[delivery],$$,"{'address1': '4163 W 5th St', 'address2': '', ...",+12133839499,(213) 383-9499,568.284344
4,iRjnWYY5oVJTaZ5teU1vOQ,king-chang-la-los-angeles,King Chang - LA,https://s3-media3.fl.yelpcdn.com/bphoto/FGJWXO...,False,https://www.yelp.com/biz/king-chang-la-los-ang...,283,"[{'alias': 'korean', 'title': 'Korean'}, {'ali...",5.0,"{'latitude': 34.06329, 'longitude': -118.30372}","[delivery, restaurant_reservation, pickup]",NaN,"{'address1': '3732 W 6th St', 'address2': None...",+12133185250,(213) 318-5250,14.161470


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
964,AFdPBnkFNLAzsH6WcPuBkw,sushi-and-teri-los-angeles,Sushi & Teri,https://s3-media2.fl.yelpcdn.com/bphoto/oZgYb6...,False,https://www.yelp.com/biz/sushi-and-teri-los-an...,385,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",3.0,"{'latitude': 34.0490383, 'longitude': -118.239...",[delivery],$,"{'address1': '116 Japanese Village Plz Mall', ...",+12136878368,(213) 687-8368,6112.604442
965,-7C4drrqJxJgn5Lh-VLakg,gabriellas-new-york-city-pizza-glendale,Gabriella's Pizza,https://s3-media3.fl.yelpcdn.com/bphoto/r4lBX-...,False,https://www.yelp.com/biz/gabriellas-new-york-c...,27,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",2.5,"{'latitude': 34.146481, 'longitude': -118.254393}","[delivery, pickup]",NaN,"{'address1': '107 E Broadway', 'address2': '',...",+17472406758,(747) 240-6758,10287.026095
966,S8A4xHxDFiklFowZzukKlw,ono-hawaiian-bbq-culver-city-2,Ono Hawaiian BBQ,https://s3-media1.fl.yelpcdn.com/bphoto/MX70jp...,False,https://www.yelp.com/biz/ono-hawaiian-bbq-culv...,94,"[{'alias': 'hawaiian', 'title': 'Hawaiian'}, {...",3.0,"{'latitude': 33.985973, 'longitude': -118.3932}","[delivery, pickup]",$,"{'address1': '6000 Sepulveda Blvd', 'address2'...",+13103906888,(310) 390-6888,11916.550669
967,FgBErSd-qVUvyR7TzwsFLg,green-truck-culver-city-4,Green Truck,https://s3-media2.fl.yelpcdn.com/bphoto/lTqMpL...,False,https://www.yelp.com/biz/green-truck-culver-ci...,365,"[{'alias': 'foodtrucks', 'title': 'Food Trucks...",3.5,"{'latitude': 34.0245873609151, 'longitude': -1...",[delivery],$$,"{'address1': '9040 Lindblade St', 'address2': ...",+13102040477,(310) 204-0477,9101.674164
968,g1K4Nr5zrIYxWu57Uq3bog,california-roll-and-sushi-culver-city,California Roll & Sushi,https://s3-media1.fl.yelpcdn.com/bphoto/NBgY6_...,False,https://www.yelp.com/biz/california-roll-and-s...,165,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",3.0,"{'latitude': 33.982834269309, 'longitude': -11...","[delivery, pickup]",$$,"{'address1': '6251 Bristol Pkwy', 'address2': ...",+13103377775,(310) 337-7775,12123.651564


In [35]:
# sorting the values by 
final_df.sort_values(by=['rating'], ascending=False)

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
248,daz9CaOzDgVBIrmdLXSm0A,maru-pit-stop-los-angeles-2,Maru Pit Stop,https://s3-media3.fl.yelpcdn.com/bphoto/O1BUlk...,False,https://www.yelp.com/biz/maru-pit-stop-los-ang...,385,"[{'alias': 'asianfusion', 'title': 'Asian Fusi...",5.0,"{'latitude': 34.14245333, 'longitude': -118.22...","[delivery, pickup]",$$,"{'address1': '2750 Colorado Blvd', 'address2':...",+13233166312,(323) 316-6312,11354.814744
85,Nb6Qle4iE3eQsUe_rAR6EQ,assa-dakbal-los-angeles-2,Assa Dakbal,https://s3-media3.fl.yelpcdn.com/bphoto/m9QEqH...,False,https://www.yelp.com/biz/assa-dakbal-los-angel...,19,"[{'alias': 'korean', 'title': 'Korean'}, {'ali...",5.0,"{'latitude': 34.061451281250044, 'longitude': ...",[],NaN,"{'address1': '3950 Wilshire Blvd', 'address2':...",+12133409424,(213) 340-9424,870.895715
165,d19eaXQrU2_ouSsaeARGeA,cho-dang-tofu-los-angeles,Cho Dang Tofu,https://s3-media1.fl.yelpcdn.com/bphoto/n8psf0...,False,https://www.yelp.com/biz/cho-dang-tofu-los-ang...,1,"[{'alias': 'korean', 'title': 'Korean'}, {'ali...",5.0,"{'latitude': 34.068589952986635, 'longitude': ...",[],NaN,"{'address1': '301 S Western Ave', 'address2': ...",+12133222562,(213) 322-2562,778.509039
394,ahjbni41lzR7LKiXGcuevQ,ellie-julie-los-angeles-3,Ellie Julie,https://s3-media2.fl.yelpcdn.com/bphoto/aRsdO5...,False,https://www.yelp.com/biz/ellie-julie-los-angel...,20,"[{'alias': 'chickenshop', 'title': 'Chicken Sh...",5.0,"{'latitude': 34.0551603, 'longitude': -118.308...",[],NaN,"{'address1': '928 S Western Ave', 'address2': ...",+12139087293,(213) 908-7293,1011.984289
281,iEO-a0_0EtvOJQZTGNt0xg,k-surf-and-turf-hollywood-los-angeles,K-Surf & Turf Hollywood,https://s3-media2.fl.yelpcdn.com/bphoto/A7pPMw...,False,https://www.yelp.com/biz/k-surf-and-turf-holly...,19,"[{'alias': 'bbq', 'title': 'Barbeque'}, {'alia...",5.0,"{'latitude': 34.101780737975275, 'longitude': ...","[delivery, pickup]",$$,"{'address1': '6655 Hollywood Blvd', 'address2'...",+13236457366,(323) 645-7366,5135.504777
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
935,Uzn8afUO48x_4QuF_Gd8FA,lily-mae-s-hot-chicken-ventura-blvd-studio-city,Lily Mae’s Hot Chicken - Ventura Blvd,,False,https://www.yelp.com/biz/lily-mae-s-hot-chicke...,1,"[{'alias': 'fooddeliveryservices', 'title': 'F...",1.0,"{'latitude': 34.140284, 'longitude': -118.375594}","[delivery, pickup]",NaN,"{'address1': '11288 Ventura Blvd', 'address2':...",+18185794099,(818) 579-4099,10799.249267
886,BVnMCBAXWPCBmeFxXbN0kQ,wings-out-los-angeles-2,Wings Out,https://s3-media4.fl.yelpcdn.com/bphoto/tJb4tZ...,False,https://www.yelp.com/biz/wings-out-los-angeles...,2,"[{'alias': 'fooddeliveryservices', 'title': 'F...",1.0,"{'latitude': 33.97845, 'longitude': -118.3926}","[delivery, pickup]",NaN,"{'address1': '6081 Center Drive', 'address2': ...",,,12513.034737
808,yuYXwVemY7ulKlQkF-npMQ,fck-gluten-los-angeles-3,Fck Gluten,https://s3-media1.fl.yelpcdn.com/bphoto/2Sk6dQ...,False,https://www.yelp.com/biz/fck-gluten-los-angele...,1,"[{'alias': 'fooddeliveryservices', 'title': 'F...",1.0,"{'latitude': 34.082282, 'longitude': -118.3093...","[pickup, delivery]",NaN,"{'address1': '615 N Western Ave', 'address2': ...",+13234571343,(323) 457-1343,2152.276404
916,DIQ8lIshyd7zk7oZrZOp0Q,house-of-fried-chicken-los-angeles-4,House of Fried Chicken,,False,https://www.yelp.com/biz/house-of-fried-chicke...,1,"[{'alias': 'fooddeliveryservices', 'title': 'F...",1.0,"{'latitude': 34.140284, 'longitude': -118.375594}","[pickup, delivery]",NaN,"{'address1': '11288 Ventura Blvd', 'address2':...",+18185794099,(818) 579-4099,10799.249267


# Data Cleaning

In [38]:
# check for duplicate results
final_df.duplicated(subset='id').sum()

1

In [39]:
## Drop duplicate ids and confirm there are no more duplicates
final_df = final_df.drop_duplicates(subset='id')
final_df.duplicated(subset='id').sum()

0

In [40]:
# save the final results to a compressed csv
final_df.to_csv('Data/final_results_LA_korean.csv.gz', compression='gzip',index=False)

In [50]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 968 entries, 0 to 968
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             968 non-null    object 
 1   alias          968 non-null    object 
 2   name           968 non-null    object 
 3   is_closed      968 non-null    bool   
 4   url            968 non-null    object 
 5   review_count   968 non-null    int64  
 6   categories     968 non-null    object 
 7   rating         968 non-null    float64
 8   coordinates    968 non-null    object 
 9   transactions   968 non-null    object 
 10  price          772 non-null    object 
 11  location       968 non-null    object 
 12  phone          968 non-null    object 
 13  display_phone  968 non-null    object 
 14  distance       968 non-null    float64
dtypes: bool(1), float64(2), int64(1), object(11)
memory usage: 114.4+ KB


In [51]:
final_df.head()

,id,alias,name,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,h1R2iKYdm2lwukzMJvJqDw,hangari-kalguksu-los-angeles-4,Hangari Kalguksu,False,https://www.yelp.com/biz/hangari-kalguksu-los-...,2385,"[{'alias': 'korean', 'title': 'Korean'}, {'ali...",4.5,"{'latitude': 34.0628602582049, 'longitude': -1...","[pickup, delivery]",$$,"{'address1': '3470 W 6th St', 'address2': 'Ste...",+12133882326,(213) 388-2326,584.192122
1,qAAoilHU25Qr45FOQlA19g,kang-ho-dong-baekjeong-los-angeles-4,Kang Ho-dong Baekjeong,False,https://www.yelp.com/biz/kang-ho-dong-baekjeon...,4902,"[{'alias': 'bbq', 'title': 'Barbeque'}, {'alia...",4.5,"{'latitude': 34.06375298900917, 'longitude': -...","[delivery, pickup]",$$$,"{'address1': '3465 W 6th St', 'address2': 'Ste...",+12133849678,(213) 384-9678,607.771480
2,WO12S4gvOYkYF8_70RWrQA,haneuem-los-angeles-3,HanEuem,False,https://www.yelp.com/biz/haneuem-los-angeles-3...,190,"[{'alias': 'korean', 'title': 'Korean'}, {'ali...",4.5,"{'latitude': 34.06427634725579, 'longitude': -...","[delivery, pickup]",NaN,"{'address1': '539 S Western Ave', 'address2': ...",+12133888988,(213) 388-8988,527.290392
3,uzAbw27XQTXTivjgf2bN2w,han-bat-sul-lung-tang-los-angeles-2,Han Bat Sul Lung Tang,False,https://www.yelp.com/biz/han-bat-sul-lung-tang...,2666,"[{'alias': 'korean', 'title': 'Korean'}, {'ali...",4.5,"{'latitude': 34.065414, 'longitude': -118.3095...",[delivery],$$,"{'address1': '4163 W 5th St', 'address2': '', ...",+12133839499,(213) 383-9499,568.284344
4,iRjnWYY5oVJTaZ5teU1vOQ,king-chang-la-los-angeles,King Chang - LA,False,https://www.yelp.com/biz/king-chang-la-los-ang...,283,"[{'alias': 'korean', 'title': 'Korean'}, {'ali...",5.0,"{'latitude': 34.06329, 'longitude': -118.30372}","[delivery, restaurant_reservation, pickup]",NaN,"{'address1': '3732 W 6th St', 'address2': None...",+12133185250,(213) 318-5250,14.161470
